In [138]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
import pandas as pd
import datetime as dt

In [140]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

ModuleNotFoundError: No module named 'fastai.structured'

In [115]:
??proc_df

Object `proc_df` not found.


In [8]:
path = "data/sapflow/"

In [72]:
weather_csv = pd.read_csv(f'{path}all_weather.txt', 
                          low_memory=False, parse_dates=[["Date","Time"]], encoding='ISO-8859-1')

sapflow_csv = pd.read_csv(f'{path}all_sapflow.txt', 
                          low_memory=False, parse_dates=[["Date","Time"]], encoding='ISO-8859-1')

In [73]:
weather_csv["Date_Time"] = weather_csv["Date_Time"].dt.strftime('%d/%m/%Y %H:%M')
sapflow_csv["Date_Time"] = sapflow_csv["Date_Time"].dt.strftime('%d/%m/%Y %H:%M')

In [74]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [20]:
display_all(weather_csv)

,Date_Time,Solar Radiation (W/m^2),Precipitation (mm),Strikes (),Strike Distance (km),Wind Speed (m/s),Wind Direction (deg),Gust Wind Speed (m/s),Air Temperature (deg.C),Vapor Pressure (kPa),Atmospheric Pressure (kPa),Raw RH (RH),xOrientation (deg),yOrientation (deg),Corrected RH (%),Evapotranspiration (mm),VPD (kPa),Rain Fall (mm),Battery Voltage (V),Battery Temperature (deg.C),External Supply (V),Diagnostic Message,Unnamed: 23
0,14/03/2018 05:10,91.0,0.000,0.0,0.0,2.579,355.700,10.710,-0.200,0.340,99.389,0.558,-1.399,-0.699,44.640,0.055,0.335,0.0,3.900,18.474,0.000,NaN,NaN
1,14/03/2018 05:20,137.0,0.000,0.0,0.0,3.410,26.100,5.699,-2.000,0.409,99.419,0.777,-0.800,-0.200,62.240,0.049,0.215,0.0,4.047,17.781,20.705,NaN,NaN
2,14/03/2018 05:30,643.0,0.000,0.0,0.0,2.960,41.700,5.409,-0.899,0.430,99.389,0.753,-1.100,-0.400,60.240,0.198,0.235,0.0,4.152,18.397,6.389,NaN,NaN
3,14/03/2018 05:40,294.0,0.000,0.0,0.0,2.880,6.400,4.849,-0.699,0.449,99.410,0.778,-1.100,-0.200,62.319,0.099,0.224,0.0,4.152,18.667,20.356,NaN,NaN
4,14/03/2018 05:50,306.0,0.000,0.0,0.0,5.019,9.399,9.170,-1.799,0.439,99.410,0.818,-1.100,-0.200,65.440,0.080,0.200,0.0,4.032,17.511,6.396,NaN,NaN
5,14/03/2018 06:00,211.0,0.000,0.0,0.0,3.759,12.399,7.019,-2.900,0.419,99.419,0.856,-1.200,-0.300,68.479,0.058,0.178,0.0,4.080,16.701,6.433,NaN,NaN
6,14/03/2018 06:10,249.0,0.000,0.0,0.0,2.869,18.799,5.039,-2.299,0.419,99.419,0.810,-1.100,-0.200,64.879,0.077,0.200,0.0,4.056,17.511,6.386,NaN,NaN
7,14/03/2018 06:20,149.0,0.000,0.0,0.0,3.549,32.299,7.590,-1.799,0.419,99.410,0.782,-1.100,-0.100,62.640,0.052,0.215,0.0,4.080,16.509,6.479,NaN,NaN
8,14/03/2018 06:30,659.0,0.000,0.0,0.0,3.579,21.000,6.630,-1.000,0.430,99.360,0.754,-1.000,-0.200,60.400,0.189,0.233,0.0,4.176,17.472,20.313,NaN,NaN
9,14/03/2018 06:40,616.0,0.000,0.0,0.0,4.289,19.600,10.449,-0.400,0.430,99.360,0.717,-0.899,-0.100,57.359,0.176,0.256,0.0,4.176,18.706,20.169,NaN,NaN


In [75]:
weather_csv = weather_csv[["Date_Time", "Solar Radiation (W/m^2)", "Air Temperature (deg.C)", "Corrected RH (%)", "Wind Speed (m/s)"]]
weather_csv.rename(columns = {"Date_Time":"datetime", "Solar Radiation (W/m^2)":"solar_rad","Air Temperature (deg.C)":"air_temp","Corrected RH (%)":"RH","Wind Speed (m/s)":"wind_speed"}, inplace=True)

In [76]:
sapflow_csv = sapflow_csv[["Date_Time", "Uncorrected Out (cm/hr)", "Uncorrected In (cm/hr)"]]
sapflow_csv.rename(columns ={"Date_Time":"datetime", "Uncorrected Out (cm/hr)":"outer_sapflow", "Uncorrected In (cm/hr)":"inner_sapflow"}, inplace=True)

In [77]:
merged=pd.merge(weather_csv, sapflow_csv, on='datetime', how='inner')

merged=merged.drop_duplicates()

In [41]:
display_all(merged)

,datetime,solar_rad,air_temp,RH,wind_speed,outer_sapflow,inner_sapflow
0,21/03/2018 19:50,0.0,-0.100,44.559,1.839,-3.620,-2.093
1,21/03/2018 20:00,0.0,-0.400,45.840,1.570,-3.435,-2.186
2,21/03/2018 20:10,0.0,-0.500,46.479,1.179,-3.077,-1.900
3,21/03/2018 20:20,0.0,-0.600,47.360,1.660,-1.704,-0.531
4,21/03/2018 20:30,0.0,-0.800,48.319,1.820,0.038,1.317
5,21/03/2018 20:40,0.0,-0.800,48.560,2.319,0.998,1.872
6,21/03/2018 20:50,0.0,-1.000,49.680,1.679,0.678,1.713
7,21/03/2018 21:00,0.0,-1.200,50.479,1.750,0.214,1.353
8,21/03/2018 21:10,0.0,-1.299,51.599,1.660,-0.156,1.083
9,21/03/2018 21:20,0.0,-1.500,52.319,1.500,-0.449,0.844


In [78]:
def label_time (row):
    hour = dt.datetime.strptime(row['datetime'],'%d/%m/%Y %H:%M').hour
    if hour >= 6 and hour < 12:
        return 'morning'
    if hour >= 12 and hour < 17:
        return 'afternoon'
    if hour >= 17 and hour < 20:
        return 'evening'
    if hour >= 20 and hour <= 24:
        return 'night'
    if hour >= 0 and hour < 6:
        return 'night'

In [ ]:
#merged.apply (label_time, axis=1)

In [79]:
merged['time_of_day'] = merged.apply (label_time, axis=1)

In [71]:
display_all(merged)

,datetime,solar_rad,air_temp,RH,wind_speed,outer_sapflow,inner_sapflow,time_of_day
0,21/03/2018 19:50,0.0,-0.100,44.559,1.839,-3.620,-2.093,evening
1,21/03/2018 20:00,0.0,-0.400,45.840,1.570,-3.435,-2.186,night
2,21/03/2018 20:10,0.0,-0.500,46.479,1.179,-3.077,-1.900,night
3,21/03/2018 20:20,0.0,-0.600,47.360,1.660,-1.704,-0.531,night
4,21/03/2018 20:30,0.0,-0.800,48.319,1.820,0.038,1.317,night
5,21/03/2018 20:40,0.0,-0.800,48.560,2.319,0.998,1.872,night
6,21/03/2018 20:50,0.0,-1.000,49.680,1.679,0.678,1.713,night
7,21/03/2018 21:00,0.0,-1.200,50.479,1.750,0.214,1.353,night
8,21/03/2018 21:10,0.0,-1.299,51.599,1.660,-0.156,1.083,night
9,21/03/2018 21:20,0.0,-1.500,52.319,1.500,-0.449,0.844,night


In [80]:
df_outer = merged.drop("inner_sapflow", axis=1)
df_inner = merged.drop("outer_sapflow", axis=1)

In [83]:
df_outer = df_outer[pd.to_numeric(df_outer['outer_sapflow'], errors='coerce').notnull()]
df_inner = df_inner[pd.to_numeric(df_inner['inner_sapflow'], errors='coerce').notnull()]

In [94]:
df_outer['outer_sapflow'] = pd.to_numeric(df_outer['outer_sapflow'])
df_inner['inner_sapflow'] = pd.to_numeric(df_inner['inner_sapflow'])

In [96]:
outer_min = df_outer['outer_sapflow'].min()
inner_min = df_inner['inner_sapflow'].min()

In [103]:
df_outer['new_sapflow'] = df_outer.apply(lambda row: row['outer_sapflow'] + abs(outer_min), axis=1)
df_inner['new_sapflow'] = df_inner.apply(lambda row: row['inner_sapflow'] + abs(inner_min), axis=1)

Use to save csv files

In [104]:
merged.to_csv(f'{path}merged.csv', index =False)
df_outer.to_csv(f'{path}df_outer.csv', index =False)
df_inner.to_csv(f'{path}df_inner.csv', index =False)